<a href="https://colab.research.google.com/github/revanth7667/STAT-663_Codes/blob/main/Notebooks/ResNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
!pip install datasets

In [ ]:
%%capture
#Importing the Required Pacakges and Libraries

#Sklearn
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

#Dataset
from datasets import load_dataset

#tensorflow
import tensorflow as tf
from tensorflow.keras.applications.resnet50 import preprocess_input

#other
import matplotlib.pyplot as plt
import numpy as np
import time
import warnings

#Suppressing Warnings
warnings.filterwarnings('ignore')

In [ ]:
# loading the Dataset and viewing it's structure
dataset = load_dataset("cifar10")
dataset

In [ ]:
# Viewing Sample Image from Training Data
element = next(iter(dataset["train"]))
plt.imshow(element["img"])
plt.title(element["label"])

## Data Preprocessing

The input image data is converted into numpy arrays and reszied to 224x224x3. The images are then normalized to have a mean of 0 and standard deviation of 1. The labels are one-hot encoded.

### Training Data

In [ ]:
X_train = np.array(
    [(np.array(i)[None, ...] / 255.0).flatten() for i in dataset["train"]["img"][:]]
)  # .reshape(-1, 32 * 32 * 3)
y_train = np.array(dataset["train"]["label"][:])

### Test Data

In [ ]:
X_test = np.array(
    [(np.array(i)[None, ...] / 255.0).flatten() for i in dataset["test"]["img"][:]]
)  # .reshape(-1, 32 * 32 * 3)
y_test = np.array(dataset["test"]["label"][:])

## Model Performance without ResNet

The data from the previous step is used to train a simple Random Forest model and logistic regression model to get a baseline performance.

The metricts used to evaluate the model are:
- Accuracy
- Time to Train and Predict
- Confusion Matrix

### Decision Trees

In [ ]:
# Initializing the Model
model_rf_raw = RandomForestClassifier(n_estimators=100)

# Fitting the Model
t = time.time()
model_rf_raw.fit(X_train, y_train)
print(f"Training Time: {time.time() - t:.2f} Seconds")

In [ ]:
# Predicting the Test Data
t = time.time()
y_pred = model_rf_raw.predict(X_test)
print(f"Prediction Time: {time.time() - t:0.2f} Seconds")

In [ ]:
# Model Evaluation
print(f"Accuracy: {accuracy_score(y_test, y_pred):.2%}")
print(f"Confusion Matrix: \n{confusion_matrix(y_test, y_pred)}")

### Logistic Regression

In [ ]:
# Initializing the Model
model_lr_raw = LogisticRegression(
    max_iter=200, multi_class="multinomial", solver="lbfgs", random_state=42
)

# Fitting the Model
t = time.time()
model_lr_raw.fit(X_train, y_train)
print(f"Training Time: {time.time() - t:0.2f} Seconds")

In [ ]:
t = time.time()
y_pred = model_lr_raw.predict(X_test)
print(f"Prediction Time: {time.time() - t:0.2f} Seconds")

In [ ]:
# Model Evaluation
print(f"Accuracy: {accuracy_score(y_test, y_pred):.2%}")
print(f"Confusion Matrix: \n{confusion_matrix(y_test, y_pred)}")

In [ ]:
assert 1 == 0

## ResNet Optimized

### Embedding the Data using ResNet50

In [ ]:
# Load a pretrained model
model = tf.keras.applications.resnet50.ResNet50(
    include_top=False,  # take any input shape, not just 224x224
    weights="imagenet",  # use the imagenet pretrained model
    pooling="avg",  # use global average pooling to get a 2048 vector from our image
)

In [ ]:
%%capture
def preprocess(img):
    processed_img = preprocess_input(  # ResNet has its own preprocesser
        tf.image.resize(  # Images need to be in (224, 224) dimension
            np.array(img), (224, 224)  # Turn PIL image object into array
        )
    )[None]
    return processed_img


# use resnet to get the embeddings
X_train_resnet = np.array(
    [model.predict(preprocess(i)) for i in dataset["train"]["img"][:]]
).reshape(-1, 2048)
X_test_resnet = np.array(
    [model.predict(preprocess(i)) for i in dataset["test"]["img"][:]]
).reshape(-1, 2048)

In [ ]:
# Viewing 1 sample embedding
plt.plot(X_train_resnet[0].flatten())

### Decision Trees

In [ ]:
# Initializing the Model
model_rf_resnet = RandomForestClassifier(n_estimators=100)

# Fitting the Model
t = time.time()
model_rf_resnet.fit(X_train_resnet, y_train)
print(f"Training Time: {time.time() - t:.2f} Seconds")

In [ ]:
# predicting the test data
t = time.time()
y_pred = model_rf_resnet.predict(X_test_resnet)
print(f"Prediction Time: {time.time() - t:0.2f} Seconds")

In [ ]:
# Model Evaluation
print(f"Accuracy: {accuracy_score(y_test, y_pred):.2%}")
print(f"Confusion Matrix: \n{confusion_matrix(y_test, y_pred)}")

### Logistic Regression

In [ ]:
# Initializing the Model
model_lr_resnet = LogisticRegression(
    max_iter=200, multi_class="multinomial", solver="lbfgs", random_state=42
)

# Fitting the Model
t = time.time()
model_lr_resnet.fit(X_train_resnet, y_train)
print(f"Training Time: {time.time() - t:0.2f} Seconds")

In [ ]:
# predicting the test data
t = time.time()
y_pred = model_lr_resnet.predict(X_test_resnet)
print(f"Prediction Time: {time.time() - t:0.2f} Seconds")

In [ ]:
# Model Evaluation
print(f"Accuracy: {accuracy_score(y_test, y_pred):.2%}")
print(f"Confusion Matrix: \n{confusion_matrix(y_test, y_pred)}")

## Results
The time to embed the data using ResNet50 was higher than the time to embed normally. however, this led to both the models showing an improvement in terms of accuray and time to train and predict when the data was embedded using ResNet50. The confusion matrix also showed a better performance.
